In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Convolution2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
## resize all the images to this 
IMAGE_SIZE = [224,224]

train_path = ('/content/drive/MyDrive/Collab Notebooks/Neural_Network/Malaria Disease/Train')
valid_path = ('/content/drive/MyDrive/Collab Notebooks/Neural_Network/Malaria Disease/Test')

In [ ]:
mobilnet = VGG19(input_shape= IMAGE_SIZE + [3], weights='imagenet',include_top=False)


80142336/80134624 [==============================] - 1s 0us/step


In [ ]:
#don't train the existing weights
for layer in mobilnet.layers:
  layer.trainable = False

In [ ]:
## useful for getting number of output classes

folders = glob('/content/drive/MyDrive/Collab Notebooks/Neural_Network/Malaria Disease/Train/*')

In [ ]:
## our layers - you can add more if you want
x = Flatten()(mobilnet.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

#create a model

model = Model(inputs = mobilnet.input, outputs= prediction)

In [ ]:
## view the structure of the model

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
# tell the model what cost and optimization method to use

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [ ]:
### Use the Image Data generator to import the Images from the dataset

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
## Make sure you provide the same target size as initialized for the image size

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Collab Notebooks/Neural_Network/Malaria Disease/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 415 images belonging to 2 classes.


In [ ]:
training_set

In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Collab Notebooks/Neural_Network/Malaria Disease/Test',
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 134 images belonging to 2 classes.


In [ ]:
## fit the model
r = model.fit_generator(
    training_set,
    validation_data = test_set,
    epochs = 50,
    steps_per_epoch = len(training_set),
    validation_steps = len(test_set)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
13/13 [==============================] - 335s 25s/step - loss: 1.8399 - accuracy: 0.4904 - val_loss: 1.9663 - val_accuracy: 0.3433
Epoch 2/50
13/13 [==============================] - 310s 24s/step - loss: 0.8928 - accuracy: 0.5754 - val_loss: 0.5213 - val_accuracy: 0.7463
Epoch 3/50
13/13 [==============================] - 314s 25s/step - loss: 0.4227 - accuracy: 0.8139 - val_loss: 0.5547 - val_accuracy: 0.6940
Epoch 4/50
13/13 [==============================] - 311s 24s/step - loss: 0.3839 - accuracy: 0.8230 - val_loss: 0.3980 - val_accuracy: 0.8060
Epoch 5/50
13/13 [==============================] - 313s 25s/step - loss: 0.3265 - accuracy: 0.8668 - val_loss: 0.3821 - val_accuracy: 0.8358
Epoch 6/50
13/13 [==============================] - 311s 24s/step - loss: 0.3052 - accuracy: 0.9040 - val_loss: 0.3782 - val_accuracy: 0.8358
Epoch 7/50
13/13 [==============================] - 313s 25s/step - loss: 0.2818 - accuracy: 0.9061 - val_loss: 0.4697 - val_accuracy: 0.7612
Epoch 

In [ ]:
#plot the loss
plt.plot(r.history['loss'], label ='train_loss')
plt.plot(r.history['val_loss'], label = 'val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
## plot the accuracy
plt.plot(r.history['accuracy'], label='train_acc')
plt.plot(r.history['val_accuracy'], label = 'val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
## save it as a h5 file

from tensorflow.keras.models import load_model

model.save('modal_vgg19.h5')

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
y_pred

In [ ]:
y_pred = np.argmax(y_pred, axis = 1)

In [ ]:
y_pred

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model = load_model('modal_vgg19.h5')

In [ ]:
img = image.load_img('/content/drive/MyDrive/Collab Notebooks/Neural_Network/Malaria Disease/Test/Uninfected/2.png',target_size = (224, 224))

In [ ]:
x = image.img_to_array(img)
x

In [ ]:
x.shape

In [ ]:
x = x/255

In [ ]:
x = np.expand_dims(x, axis=0)
img_data = preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)

In [ ]:
a = np.argmax(model.predict(img_data), axis=1)

In [ ]:
if (a == 1):
  print("Uninfected")
else:
  print("Infected")